In [ ]:
# LIVIAS KODE

In [164]:
# attach lanbels to data
labels = {
    'sblack': 'Black',
    'shisp': 'Hispanic',
    'swhite': 'White',
    'sother': 'Other race',
    'sage': 'Age',
    'sempl': 'Employed last week or not',
    'smale': 'Male',
    'spop': 'Population size of civilian\'s address',
    'sincome': 'Income (categorical)',
    'sbehavior': 'Behavior of civilian in encounter',
    'omajwhite': 'Officer unit majorly white',
    'omajother': 'Officer unit majorly other',
    'omajblack': 'Officer unit majorly black',
    'omajhisp': 'Officer unit majorly hispanic',
    'osplit': 'Officer unit split race',
    'daytime': 'Time of encounter',
    'year': 'Year',
    'inctype_lin': 'Incident type',
    'anyuseofforce_coded': 'Any use of force by officer'}

data_labeled = data.rename(columns=labels)
data_labeled.columns

Index(['Black', 'Hispanic', 'White', 'Other race', 'Male', 'Age',
       'Employed last week or not', 'Income (categorical)',
       'Population size of civilian's address', 'Time of encounter',
       'Incident type', 'Officer unit majorly black',
       'Officer unit majorly hispanic', 'Officer unit majorly white',
       'Officer unit majorly other', 'Officer unit split race',
       'Behavior of civilian in encounter', 'Year',
       'Any use of force by officer'],
      dtype='object')

In [165]:
# table with summary statistics conditional on race and interaction
# a. make columns for each race and full sample
full_sample = data
white_only = data[data['swhite'] == 1]
black_only = data[data['sblack'] == 1]
hispanic_only = data[data['shisp'] == 1]
other_only = data[data['sother'] == 1 ]
groups = [full_sample, white_only, black_only, hispanic_only, other_only]
group_names = ['Full sample', 'White only', 'Black only', 'Hispanic only', 'Other only']
variables = ['sage','sempl', 'smale','sincome', 'spop', 'sbehavior', 'omajwhite', 
             'omajblack', 'omajhisp', 'omajother', 'osplit', 'daytime', 
             'inctype_lin', 'anyuseofforce_coded']
variables_labeled = [labels[var] for var in variables]
#####################################################
# create empty dataframe to store summary statistics
summary_stats = pd.DataFrame(columns=group_names, index=variables_labeled)
# append two columns for p-values from t-tests between white vs black and hispanic vs white
summary_stats['p-value (Black vs White)'] = np.nan
summary_stats['p-value (Hispanic vs White)'] = np.nan
summary_stats.index.name = 'Variable'
for i, group in enumerate(groups):
    summary_stats.iloc[:, i] = group[variables].mean().round(2) #mean
    # t-test means of black vs white and hispanic vs white
    if group_names[i] == 'Black only':
        from scipy.stats import ttest_ind
        t_stat, p_val = ttest_ind(black_only[variables], white_only[variables], equal_var=False, nan_policy='omit')
        summary_stats['p-value (Black vs White)'] = p_val.round(2)
    if group_names[i] == 'Hispanic only':
        from scipy.stats import ttest_ind
        t_stat, p_val = ttest_ind(hispanic_only[variables], white_only[variables], equal_var=False, nan_policy='omit')
        summary_stats['p-value (Hispanic vs White)'] = p_val.round(2)

#####################################################
summary_stats.to_latex('summary_stats.tex', index=True, decimal=',', float_format="%.2f")

In [167]:
# Load the dataset
dat = pd.read_csv('ppcs_cc.csv')

# Inspect distribution of the target variable
print("\nDistribution of 'anyuseofforce_coded':")
print(dat['anyuseofforce_coded'].value_counts(normalize=True))

# Inspect value counts for categorical variables
categorical_vars = ["sblack", "shisp", "swhite", "sother", "smale", "omajblack", 
                    "omajhisp", "omajwhite", "omajother", "osplit", "inctype_lin", "sbehavior"]

for var in categorical_vars:
    print(f"\nValue Counts for {var}:")
    print(dat[var].value_counts())


Distribution of 'anyuseofforce_coded':
anyuseofforce_coded
0    0.994999
1    0.005001
Name: proportion, dtype: float64

Value Counts for sblack:
sblack
0    3379
1     420
Name: count, dtype: int64

Value Counts for shisp:
shisp
0    3413
1     386
Name: count, dtype: int64

Value Counts for swhite:
swhite
1    2808
0     991
Name: count, dtype: int64

Value Counts for sother:
sother
0    3614
1     185
Name: count, dtype: int64

Value Counts for smale:
smale
1    2012
0    1787
Name: count, dtype: int64

Value Counts for omajblack:
omajblack
0    3568
1     231
Name: count, dtype: int64

Value Counts for omajhisp:
omajhisp
0    3708
1      91
Name: count, dtype: int64

Value Counts for omajwhite:
omajwhite
1    3433
0     366
Name: count, dtype: int64

Value Counts for omajother:
omajother
0    3755
1      44
Name: count, dtype: int64

Value Counts for osplit:
osplit
0    3799
Name: count, dtype: int64

Value Counts for inctype_lin:
inctype_lin
2    3641
1     158
Name: count, dtype

Table with summary statistics

In [197]:
# Declare labels    
y_lab = 'anyuseofforce_coded'
#x_lab = ['const', 'sblack', 'shisp', 'sother']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother']
x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior']

#scale age and make age squared
dat['sage'] = dat['sage'] / 10
dat['sagesq'] = dat.sage * dat.sage 

# create extra variables 
N = dat.shape[0]
dat['const'] = np.ones((N,))

# Rebuild the dataset
dat = dat[[y_lab] + x_lab].copy()

# Check for missing data
assert dat.notnull().all(axis=1).all(), 'Missing values detected. Clean your data!'



Distribution of 'anyuseofforce_coded':
anyuseofforce_coded
0    0.994999
1    0.005001
Name: proportion, dtype: float64

Value Counts for sblack:
sblack
0    3379
1     420
Name: count, dtype: int64

Value Counts for shisp:
shisp
0    3413
1     386
Name: count, dtype: int64

Value Counts for swhite:
swhite
1    2808
0     991
Name: count, dtype: int64

Value Counts for sother:
sother
0    3614
1     185
Name: count, dtype: int64

Value Counts for smale:
smale
1    2012
0    1787
Name: count, dtype: int64

Value Counts for omajblack:
omajblack
0    3568
1     231
Name: count, dtype: int64

Value Counts for omajhisp:
omajhisp
0    3708
1      91
Name: count, dtype: int64

Value Counts for omajwhite:
omajwhite
1    3433
0     366
Name: count, dtype: int64

Value Counts for omajother:
omajother
0    3755
1      44
Name: count, dtype: int64

Value Counts for osplit:
osplit
0    3799
Name: count, dtype: int64

Value Counts for inctype_lin:
inctype_lin
2    3641
1     158
Name: count, dtype

In [170]:
# Extract y and X
y = dat[y_lab].values
x = dat[x_lab].values
K = x.shape[1]

print(K)
print(np.shape(x))

16
(3799, 16)


Number of 1s in 'anyuseofforce_coded': 19


## Estimate using Probit

In [199]:
# Initialize starting values
import w8_probit as probit
import w8_estimation as est
import w8_logit as logit

theta0 = probit.starting_values(y, x)

# Estimate model with probit
probit_results = est.estimate(probit.q, theta0, y, x, cov_type='Sandwich')

16
(3799, 16)


In [174]:
probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')
probit_tab

Optimizer succeeded after 159 iter. (2958 func. evals.). Final criterion:  0.02225.
Probit, y = anyuseofforce_coded


,theta,se,t
const,-2.6161,0.6954,-3.7622
sblack,0.2042,0.2988,0.6833
shisp,0.4162,0.2382,1.7468
sother,0.1051,0.4384,0.2397
smale,0.5619,0.2102,2.6736
sempl,-0.5001,0.2205,-2.2676
sincome,0.1022,0.1243,0.8216
spop,0.1988,0.0747,2.6626
sage,0.4861,0.3719,1.3072
sagesq,-0.0791,0.0499,-1.5856


## Estimate using Logit

In [203]:
logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')
logit_tab

Optimizer succeeded after 159 iter. (2958 func. evals.). Final criterion:  0.02225.
Probit, y = anyuseofforce_coded


,theta,se,t
const,-2.6161,0.6954,-3.7622
sblack,0.2042,0.2988,0.6833
shisp,0.4162,0.2382,1.7468
sother,0.1051,0.4384,0.2397
smale,0.5619,0.2102,2.6736
sempl,-0.5001,0.2205,-2.2676
sincome,0.1022,0.1243,0.8216
spop,0.1988,0.0747,2.6626
sage,0.4861,0.3719,1.3072
sagesq,-0.0791,0.0499,-1.5856


## Average partial effects

### Probit

In [205]:
# Estimating the average partial effects using the probit
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]  
labels = ['sblack', 'shispanic', 'sother'] 
probit.properties(x, probit_results['theta'],print_out = True,se=True,indices=indices, labels = labels)

Optimizer succeeded after 203 iter. (3604 func. evals.). Final criterion:  0.02239.
Logit, y = anyuseofforce_coded


,theta,se,t
const,-5.6105,1.9349,-2.8996
sblack,0.4362,0.8092,0.5390
shisp,0.9404,0.6142,1.5312
sother,-0.0945,1.3081,-0.0723
smale,1.1543,0.5733,2.0135
sempl,-1.1609,0.6149,-1.8880
sincome,0.2021,0.3418,0.5914
spop,0.4812,0.1903,2.5287
sage,1.3028,1.1759,1.1079
sagesq,-0.2207,0.1652,-1.3362


### Logit

In [178]:
# Estimating the average partial effects using the logit
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]  
labels = ['sblack', 'shispanic', 'sother']  
logit.properties(x, logit_results['theta'],print_out = True,se=True,indices=indices, labels = labels)

,Estimate
sblack,0.002
shispanic,0.005
sother,-0.000


## Partial Effects

#### Defining different fixed vectors

In [207]:
#means of the regressors
print(f"{np.mean(dat['sage']):.2f}")
print(f"{np.mean(dat['sagesq']):.2f}")
print(f"{np.mean(dat['sincome']):.2f}")
print(f"{np.mean(dat['spop']):.2f}")

,Estimate
sblack,0.002
shispanic,0.005
sother,-0.000


In [212]:
# bootstrap APE for probit and logit to obtain standard errors
def bootstrap(model_func, y, x, indices, n_bootstrap=1000):
    N = len(y)
    ape_bootstrap = np.zeros((n_bootstrap, len(indices)))
    
    for b in range(n_bootstrap):
        # Resample with replacement
        sample_indices = np.random.choice(N, N, replace=True)
        y_b = y[sample_indices]
        x_b = x[sample_indices, :]
        
        # Estimate model
        theta0_b = model_func.starting_values(y_b, x_b)
        results_b = est.estimate(model_func.q, theta0_b, y_b, x_b, cov_type='Sandwich')
        
        # Compute APE
        ape_b = model_func.properties(x_b, results_b['theta'], print_out=False, se=False, indices=indices, labels=None)
        ape_bootstrap[b, :] = ape_b
    
    # Compute standard errors
    ape_se = ape_bootstrap.std(axis=0)
    
    return ape_se
# Indices for sblack, shisp, sother
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]
# Bootstrap standard errors for probit APE
# bootstrap APE for probit and logit to obtain standard errors
def bootstrap(model_module, y, x, indices, n_bootstrap=1000, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
    N = len(y)
    ape_boot = np.zeros((n_bootstrap, len(indices)))
    for b in range(n_bootstrap):
        # Resample with replacement
        sample_idx = np.random.choice(N, N, replace=True)
        y_b = y[sample_idx]
        x_b = x[sample_idx, :].copy()

        # Estimate model on bootstrap sample
        theta0_b = model_module.starting_values(y_b, x_b)
        results_b = est.estimate(model_module.q, theta0_b, y_b, x_b, cov_type='Sandwich')
        theta_b = results_b['theta']

        # baseline predicted probabilities
        p0 = model_module.G(x_b @ theta_b)

        # For each discrete change index, set that regressor to 1 and compute mean change
        for j, k in enumerate(indices):
            x_b1 = x_b.copy()
            x_b1[:, k] = 1
            p1 = model_module.G(x_b1 @ theta_b)
            ape_boot[b, j] = np.mean(p1 - p0)

    # bootstrap standard errors
    return ape_boot.std(axis=0)

# Indices for sblack, shisp, sother
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]

# Bootstrap standard errors for probit APE
probit_ape_se = bootstrap(probit, y, x, indices, n_bootstrap=1000, random_state=123)
print("\nBootstrap Standard Errors for Probit APE:")
for label, se in zip(['sblack', 'shispanic', 'sother'], probit_ape_se):
    print(f"{label}: {se:.4f}")
print("\nBootstrap Standard Errors for Probit APE:")
for label, se in zip(['sblack', 'shispanic', 'sother'], probit_ape_se):
    print(f"{label}: {se:.4f}")

Optimization terminated successfully.
         Current function value: 0.018754
         Iterations: 150
         Function evaluations: 2907
         Gradient evaluations: 171
Optimization terminated successfully.
         Current function value: 0.023547
         Iterations: 152
         Function evaluations: 3094
         Gradient evaluations: 182
Optimization terminated successfully.
         Current function value: 0.023547
         Iterations: 152
         Function evaluations: 3094
         Gradient evaluations: 182
Optimization terminated successfully.
         Current function value: 0.024108
         Iterations: 161
         Function evaluations: 3077
         Gradient evaluations: 181
Optimization terminated successfully.
         Current function value: 0.024108
         Iterations: 161
         Function evaluations: 3077
         Gradient evaluations: 181
Optimization terminated successfully.
         Current function value: 0.020382
         Iterations: 156
         Functi

Optimization terminated successfully.
         Current function value: 0.018754
         Iterations: 150
         Function evaluations: 2907
         Gradient evaluations: 171
Optimization terminated successfully.
         Current function value: 0.023547
         Iterations: 152
         Function evaluations: 3094
         Gradient evaluations: 182
Optimization terminated successfully.
         Current function value: 0.023547
         Iterations: 152
         Function evaluations: 3094
         Gradient evaluations: 182
Optimization terminated successfully.
         Current function value: 0.024108
         Iterations: 161
         Function evaluations: 3077
         Gradient evaluations: 181
Optimization terminated successfully.
         Current function value: 0.024108
         Iterations: 161
         Function evaluations: 3077
         Gradient evaluations: 181
Optimization terminated successfully.
         Current function value: 0.020382
         Iterations: 156
         Functi

LinAlgError: Singular matrix

In [180]:
# Original vector
# make vector of stereotypical white young man 
x_lab = ['const', 'sblack', 'shisp', 'sother', 
         'smale', 'sage', 'sempl', 'sincome',
         'spop', 'daytime', 'inctype_lin', 'omajblack',
         'omajhisp', 'omajother', 'sbehavior','sagesq']

x_me = np.array([1, 0, 0, 0,
                 1, 2.5, 0, 1,
                 4, 5, 1, 0, # daytime = 5 means the incident happened at night
                 0, 0, 1, 6.25]).reshape(1, -1)

pd.DataFrame(x_me, columns=x_lab, index=['x_me'])


,const,sblack,shisp,sother,smale,sage,sempl,sincome,spop,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior,sagesq
x_me,1.0,0.0,0.0,0.0,1.0,4.1,0.0,2.16,1.36,0.0,1.0,0.0,0.0,0.0,0.0,19.42


In [181]:
### BEHAVIOR = 1 ###
# Let us make a vector of the values we want to investigate
#x_me= np.array([1, 0, 0, 0, 1, 4.1, 0, 2.16, 1.36,0,1,0,0,0,1,19.42]).reshape(1, -1)
#pd.DataFrame(x_me, columns=x_lab, index=['x_behavior'])


#### Swiching race from white to black, hispanic and other

In [281]:
# make vector of stereotypical white old woman
x_lab = ['const', 'sblack', 'shisp', 'sother', 
         'smale', 'sage', 'sempl', 'sincome',
         'spop', 'daytime', 'inctype_lin', 'omajblack',
         'omajhisp', 'omajother', 'sbehavior','sagesq']

x_me = np.array([1, 0, 0, 0,
                 0, 4.5, 1, 3,
                 2, 2, 1, 0, 
                 0, 0, 1, 20.25]).reshape(1, -1)

pd.DataFrame(x_me, columns=x_lab, index=['x_me'])


,const,sblack,shisp,sother,smale,sage,sempl,sincome,spop,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior,sagesq
x_me,1.0,0.0,0.0,0.0,0.0,4.5,1.0,3.0,2.0,2.0,1.0,0.0,0.0,0.0,1.0,20.25


In [334]:
"""# the average characteristics for age, agesq, income, spop
print(f"{np.mean(dat['sage']):.2f}")
print(f"{np.mean(dat['sagesq']):.2f}")
print(f"{np.mean(dat['sincome']):.2f}")
print(f"{np.mean(dat['spop']):.2f}")
print(f"{np.mean(dat['daytime']):.2f}")

x_lab = ['const', 'sblack', 'shisp', 'sother', 
         'smale', 'sage', 'sempl', 'sincome',
         'spop', 'daytime', 'inctype_lin', 'omajblack',
         'omajhisp', 'omajother', 'sbehavior','sagesq']

x_me = np.array([1, 0, 0, 0,
                 0, 4.1, 0, 2.16,
                 1.36, 5, 1, 0, 
                 0, 0, 1, 19.42]).reshape(1, -1)

pd.DataFrame(x_me, columns=x_lab, index=['x_me'])"""

'# the average characteristics for age, agesq, income, spop\nprint(f"{np.mean(dat[\'sage\']):.2f}")\nprint(f"{np.mean(dat[\'sagesq\']):.2f}")\nprint(f"{np.mean(dat[\'sincome\']):.2f}")\nprint(f"{np.mean(dat[\'spop\']):.2f}")\nprint(f"{np.mean(dat[\'daytime\']):.2f}")\n\nx_lab = [\'const\', \'sblack\', \'shisp\', \'sother\', \n         \'smale\', \'sage\', \'sempl\', \'sincome\',\n         \'spop\', \'daytime\', \'inctype_lin\', \'omajblack\',\n         \'omajhisp\', \'omajother\', \'sbehavior\',\'sagesq\']\n\nx_me = np.array([1, 0, 0, 0,\n                 0, 4.1, 0, 2.16,\n                 1.36, 5, 1, 0, \n                 0, 0, 1, 19.42]).reshape(1, -1)\n\npd.DataFrame(x_me, columns=x_lab, index=[\'x_me\'])'

In [184]:
b_pr = probit_tab.theta.values
me_race_pr = probit.G(x_me2@b_pr) - probit.G(x_me@b_pr) 

In [320]:
gx0 = norm.pdf(x_me@b_pr)
gx2 = norm.pdf(x_me2@b_pr)

grad_d_pr = gx2*x_me2 - gx0*x_me

,const,sblack,shisp,sother,smale,sage,sempl,sincome,spop,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior,sagesq
x_me2,1.0,0.0,1.0,0.0,1.0,2.5,0.0,1.0,4.0,5.0,1.0,0.0,0.0,0.0,1.0,6.25


In [186]:
def get_se(grad, cov):
    cov_me = grad@cov@grad.T
    return np.sqrt(np.diag(cov_me))

se_d_pr = get_se(grad_d_pr, probit_results['cov'])

In [321]:
me_dict = {'Marginal Effect': me_race_pr[0],
           's.e.':            se_d_pr}
tab = pd.DataFrame(me_dict)
tab['t'] = tab['Marginal Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(6)

,Marginal Effect,s.e.,t
Var,,,
0,0.0,0.0,0.0


### Logit

In [323]:
b_lg = logit_tab.theta.values
me_race_lg = logit.G(x_me2@b_lg) - logit.G(x_me@b_lg)

In [324]:
# Compute the logistic function exponential terms for x_me2 and x_me
exp_x0_b = np.exp(-(x_me@b_lg))
exp_x2_b = np.exp(-(x_me2@b_lg))

grad_d_lg = (x_me2 * exp_x2_b)/ (1 + exp_x2_b)**2 - (x_me * exp_x0_b)/ (1 + exp_x0_b)**2

se_d_lg = get_se(grad_d_lg, logit_results['cov'])

,Marginal Effect,s.e.,t
Var,,,
0,0.003566,0.013393,0.266277


In [325]:
# find p value for marginal effects
from scipy.stats import norm
p_values = 2 * (1 - norm.cdf(np.abs(tab['t'])))
tab['p-value'] = p_values

In [326]:
tab.to_latex('me_probit_youngman_hisp.tex')

In [190]:
me_dict = {'Marginal Effect': me_race_lg[0],
           's.e.':            se_d_lg}
tab = pd.DataFrame(me_dict)
tab['t'] = tab['Marginal Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(6)

,Marginal Effect,s.e.,t
Var,,,
0,0.0,0.0,0.0


In [328]:
p_values = 2 * (1 - norm.cdf(np.abs(tab['t'])))
tab['p-value'] = p_values

In [329]:
tab.to_latex('me_logit_youngman_hisp.tex')

,Marginal Effect,s.e.,t
Var,,,
0,0.001049,0.004339,0.241665


In [338]:
files = [
    'me_probit_youngman.tex',
    'me_probit_youngman_hisp.tex',
    'me_logit_youngman.tex',
    'me_logit_youngman_hisp.tex',
    'me_probit_old_woman.tex',
    'me_probit_old_woman_hisp.tex',
    'me_logit_old_woman.tex',
    'me_logit_old_woman.tex'
]

with open('combined_table.tex', 'w') as outfile:
    outfile.write(r'\begin{table}[ht!]' + '\n')
    outfile.write(r'\centering' + '\n')
    outfile.write(r'\caption{Marginal Effects from Probit and Logit Models}' + '\n')
    outfile.write(r'\begin{tabular}{lccc}' + '\n')
    outfile.write(r'\toprule' + '\n')
    
    panels = ['Panel A: Probit, Young Man',
              'Panel B: Probit, Young Man Hispanic',
              'Panel C: Logit, young man',
              'Panel D: Logit, young man, hispanic',
              'Panel E: Probit, Old Woman',
              'Panel F: Probit, Old Woman, hispanic',
              'Panel G: Logit, Old Woman',
              'Panel H: Logit, Old Woman, hispanic']

    for panel_name, fname in zip(panels, files):
        outfile.write(r'\midrule' + '\n')
        outfile.write(r'\multicolumn{4}{c}{\textbf{' + panel_name + r'}} \\' + '\n')
        outfile.write(r'\midrule' + '\n')
        with open(fname) as infile:
            content = infile.read()
            outfile.write(content + '\n')
    
    outfile.write(r'\bottomrule' + '\n')
    outfile.write(r'\end{tabular}' + '\n')
    outfile.write(r'\end{table}' + '\n')